---
title: "Move Semantics"
author: "Quasar"
date: "2024-10-26"
categories: [C++]      
image: "cpp.jpg"
toc: true
toc-depth: 3
format:
    html:
        code-tools: true
        code-block-border-left: true
        code-annotations: below
        highlight-style: pygments
---


## Motivation for Move Semantics

To understand the basic principles of move semantics, let's look at the execution of a small piece of code. I've written a toy `Vector` class. I have also overloaded `operator+()` to support element-wise addition of two vectors. 


In [ ]:
%load_ext itikz

### Before move semantics

Assume that we have the following program:

```cpp
#include <iostream>
#include <stdexcept>
#include <initializer_list>

#define assertm(exp, msg) assert(((void)msg, exp))

template <typename T>
class Vector {
private:
    int capacity_;
    int size_;
    T* ptr_;

public:
    Vector<T>() :capacity_{ 0 }, size_{ 0 }, ptr_{ nullptr } {}
    Vector<T>(int size) : capacity_{ size }, ptr_{ new T[size] }, size_{ size } {}
    Vector<T>(int size, T data) : Vector<T>(size) {
        for (int i{ 0 }; i < size; ++i)
            ptr_[i] = data;
    }

    Vector<T>(std::initializer_list<T> list) {
        clear();
        for (const T& elem : list)
            push_back(elem);
    }

    //Destructor
    ~Vector<T>()
    {
        clear();
    }

    //Copy constructor
    Vector<T>(const Vector<T>& v)
    {
        if (this == &v)
            return;

        capacity_ = v.capacity_;
        size_ = v.size_;
        ptr_ = new T[v.size_];

        for (int i{ 0 }; i < v.size_; ++i)
            ptr_[i] = v.ptr_[i];
    }

    //Copy assignment operator
    Vector<T>& operator=(const Vector<T>& v)
    {
        if (ptr_ != nullptr)
            delete[] ptr_;

        capacity_ = v.capacity_;
        size_ = v.size_;
        ptr_ = new T[v.size_];

        for (int i{ 0 }; i < v.size_; ++i)
            ptr_[i] = v.ptr_[i];

        return *this;
    }

    T operator[](int i)
    {
        if (i < size_)
            return ptr_[i];
        else
            throw std::out_of_range("Index out of bounds.");
    }

    T operator[](int i) const
    {
        if (i < size_)
            return ptr_[i];
        else
            throw std::out_of_range("Index out of bounds.");
    }

    void reserve(int size)
    {
        if (ptr_ == nullptr)
        {
            size_ = 0;
            capacity_ = 0;
        }

        T* bufferNew = new T[size];
        unsigned int l_size = std::min(capacity_, size);
        for (int i{ 0 }; i < l_size; ++i)
        {
            bufferNew[i] = ptr_[i];
        }

        if (ptr_ != nullptr)
            delete[] ptr_;

        ptr_ = bufferNew;
        capacity_ = size;
    }

    void clear()
    {
        if (ptr_ != nullptr)
            delete[] ptr_;
        ptr_ = nullptr;
        size_ = 0;
        capacity_ = 0;
    }

    int size() const
    {
        return size_;
    }

    int capacity()
    {
        return capacity_;
    }

    void push_back(const T& elem)
    {
        if (size_ >= capacity_)
            reserve(capacity_ + 5);

        ptr_[size_++] = elem;
    }

    void pop_back()
    {
        --size_;
    }

    T front()
    {
        if (size_ > 0)
            return ptr_[0];
        else
            throw std::out_of_range("Index out of bounds.");
    }

    T back()
    {
        if (size_ > 0)
            return ptr_[size_ - 1];
        else
            throw std::out_of_range("Index out of bounds.");
    }
};

template <typename T>
Vector<T> operator+(const Vector<T>& v1, const Vector<T>& v2)
{
    if (v1.size() != v2.size())
        throw std::logic_error("Vector lengths must be equal.");
    Vector<T> result;
    for (int i{ 0 }; i < v1.size(); ++i)
        result.push_back(v1[i] + v2[i]);

    return result;
}

int main()
{
    Vector<Vector<double>> pts;
    pts.reserve(3);
    Vector<double> x{1.0, 1.0};
    std::cout << "\nCheckpoint #1";
    std::cout << "\npts->ptr_ = "<<pts.getRawPointer();
    std::cout << "\nx->ptr_ = "<<x.getRawPointer();
    pts.push_back(x);
    pts.push_back(x + x);
    pts.push_back(x);
    return 0;
}
```

[Compiler Explorer](https://godbolt.org/z/b48xxeEKM){target="_blank"}

Let us look at the individual steps of the program (inspecting both stack and the heap) when we compile this program with a C++ compiler.

First in `main`, we create the empty vector `pts` which will be used to store points in the euclidean space $\mathbf{R}^2$:

```
Vector<Vector<double>> pts;
```

which is placed on the stack as an object that has `size_ = 0`, `capacity_ = 0` and no memory allocated for elements.

Then, we call 

```
pts.reserve(3);
```

This allocates memory for `3` elements on the heap. The member `pts_->capacity_` equals `3`, `pts->size_` equals `0` and `pts_->ptr_` contains the address to heap block. The allocated memory is not initialized, because the number of elements is still `0`.

Then, we create a $2$-tuple to hold the cartesian coordinates of a point $(1.0,1.0)$. We create a `Vector<double>` initialized to `{1.0,1.0}`. Essentially, we create an object `x` on the stack with its members `x->size_ = 2`, `x->capacity_ = 5` and a pointer `x->ptr_` containing the address of newly allocated memory on the heap for `5` elements. Further, `ptr_[0]=1.0`, `ptr_[1]=1.0`. 

```
Vector<double> x{1.0, 1.0};
```

After this statement, the program has the following state: we have two objects on the stack : `pts` and `x`. Both of them have memory allocated on the heap. 

![Checkpoint #1](move_semantics_00.png)

The next step is the command to insert `x` into the `pts` vector. 

```
pts.push_back(x);
```

My toy `Vector` class is said to have value semantics, which means it creates copies of the values passed to it. As a result, we get a first element in the vector, which is a full(deep) copy of the passed value/object `x`:

![Checkpoint #2](move_semantics_01.png)

The current state is that we have a vector `pts` and two copies of `x={1.0,1.0}`, one of which is the first element in `pts`. 

Let's now look at the next statement, which creates a new temporary vector and again inserts it into the `pts` vector:

```
pts.push_back(x + x);
```

This statement is performed in three steps:

*Step 1*. We create a temporary `Vector<double>` object `x + x`. 

![Step #1](move_semantics_02.png)

*Step 2*.  `x+x` is a temporary.  The `Vector<T>::push_back(const T&)` function accepts a reference-to-`const` as an argument. Since `x+x` is a temporary, it cannot be modified and binds to a reference-to-`const`. Moreover, being a temporary object, it is likely to die soon. Referencing it extends the lifetime of the temporary x + x={2.0,2.0}. 

Now, the statement `pts_[size++] = elem` will invoke the copy-assignment operator on the yet uninitialized second element  `pts[1]` which is of type `Vector<double>`. This will force a full (deep) copy of `x + x={2.0,2.0}`. At this time, two copies of `{2.0,2.0}`  exist on the heap. One of these is assigned to pts[1]. 


In [ ]:
# | code-fold: true
# | code-summary: "Show the code"
%%itikz --temp-dir --tex-packages=tikz --tikz-libraries=arrows --implicit-standalone
\begin{tikzpicture}[x=0.75pt,y=0.75pt,yscale=-1,xscale=1]
%uncomment if require: \path (0,468); %set diagram left start at 0, and has height of 468

%Straight Lines [id:da6268999363061931] 
\draw    (179.67,22.67) -- (181,459.33) ;
%Shape: Rectangle [id:dp19854013107143653] 
\draw   (44,51) -- (168.5,51) -- (168.5,120.75) -- (44,120.75) -- cycle ;
%Shape: Rectangle [id:dp5840135846204244] 
\draw   (41.5,233) -- (166,233) -- (166,302.75) -- (41.5,302.75) -- cycle ;
%Shape: Rectangle [id:dp2555444801624851] 
\draw  [fill={rgb, 255:red, 254; green, 255; blue, 147 }  ,fill opacity=1 ] (251.5,52.03) -- (360,52.03) -- (360,121) -- (251.5,121) -- cycle ;
%Shape: Rectangle [id:dp6472270038953516] 
\draw   (248.5,275.25) -- (278,275.25) -- (278,305) -- (248.5,305) -- cycle ;
%Shape: Rectangle [id:dp2373984866383141] 
\draw   (278,275.25) -- (307.5,275.25) -- (307.5,305) -- (278,305) -- cycle ;
%Shape: Rectangle [id:dp3986501691309581] 
\draw   (307.5,275.25) -- (337,275.25) -- (337,305) -- (307.5,305) -- cycle ;
%Shape: Rectangle [id:dp5810105569962378] 
\draw   (337,275.25) -- (366.5,275.25) -- (366.5,305) -- (337,305) -- cycle ;
%Shape: Rectangle [id:dp6231112988693166] 
\draw   (366.5,275.25) -- (396,275.25) -- (396,305) -- (366.5,305) -- cycle ;
%Straight Lines [id:da9496626817348288] 
\draw    (162.5,289.03) -- (246,289.24) ;
\draw [shift={(249,289.25)}, rotate = 180.14] [fill={rgb, 255:red, 0; green, 0; blue, 0 }  ][line width=0.08]  [draw opacity=0] (8.93,-4.29) -- (0,0) -- (8.93,4.29) -- cycle    ;
%Straight Lines [id:da5923792101660912] 
\draw    (162,103.03) -- (248,103.24) ;
\draw [shift={(251,103.25)}, rotate = 180.14] [fill={rgb, 255:red, 0; green, 0; blue, 0 }  ][line width=0.08]  [draw opacity=0] (8.93,-4.29) -- (0,0) -- (8.93,4.29) -- cycle    ;
%Shape: Rectangle [id:dp8713965097719767] 
\draw   (206.5,155.75) -- (236,155.75) -- (236,185.5) -- (206.5,185.5) -- cycle ;
%Shape: Rectangle [id:dp1075315862888877] 
\draw   (236,155.75) -- (265.5,155.75) -- (265.5,185.5) -- (236,185.5) -- cycle ;
%Shape: Rectangle [id:dp07569442314741215] 
\draw   (265.5,155.75) -- (295,155.75) -- (295,185.5) -- (265.5,185.5) -- cycle ;
%Shape: Rectangle [id:dp015519266538059684] 
\draw   (295,155.75) -- (324.5,155.75) -- (324.5,185.5) -- (295,185.5) -- cycle ;
%Shape: Rectangle [id:dp8513271554424788] 
\draw   (324.5,155.75) -- (354,155.75) -- (354,185.5) -- (324.5,185.5) -- cycle ;
%Straight Lines [id:da9509465887546247] 
\draw    (273.5,120.53) -- (273.5,152.25) ;
\draw [shift={(273.5,155.25)}, rotate = 270] [fill={rgb, 255:red, 0; green, 0; blue, 0 }  ][line width=0.08]  [draw opacity=0] (8.93,-4.29) -- (0,0) -- (8.93,4.29) -- cycle    ;
%Shape: Rectangle [id:dp738771859520734] 
\draw   (43.5,335.67) -- (168,335.67) -- (168,405.42) -- (43.5,405.42) -- cycle ;
%Straight Lines [id:da8919822087320273] 
\draw    (164.5,387.7) -- (248,387.91) ;
\draw [shift={(251,387.92)}, rotate = 180.14] [fill={rgb, 255:red, 0; green, 0; blue, 0 }  ][line width=0.08]  [draw opacity=0] (8.93,-4.29) -- (0,0) -- (8.93,4.29) -- cycle    ;
%Shape: Rectangle [id:dp05261270143272889] 
\draw   (252.5,371.92) -- (282,371.92) -- (282,401.67) -- (252.5,401.67) -- cycle ;
%Shape: Rectangle [id:dp20628248058788512] 
\draw   (282,371.92) -- (311.5,371.92) -- (311.5,401.67) -- (282,401.67) -- cycle ;
%Shape: Rectangle [id:dp8880353993167662] 
\draw   (311.5,371.92) -- (341,371.92) -- (341,401.67) -- (311.5,401.67) -- cycle ;
%Shape: Rectangle [id:dp04563401463090777] 
\draw   (341,371.92) -- (370.5,371.92) -- (370.5,401.67) -- (341,401.67) -- cycle ;
%Shape: Rectangle [id:dp0706714368117467] 
\draw   (370.5,371.92) -- (400,371.92) -- (400,401.67) -- (370.5,401.67) -- cycle ;
%Straight Lines [id:da5742980804835596] 
\draw    (477.5,-73.3) -- (561,-73.09) ;
\draw [shift={(564,-73.08)}, rotate = 180.14] [fill={rgb, 255:red, 0; green, 0; blue, 0 }  ][line width=0.08]  [draw opacity=0] (8.93,-4.29) -- (0,0) -- (8.93,4.29) -- cycle    ;
%Shape: Rectangle [id:dp260318587360455] 
\draw   (323.5,204.92) -- (353,204.92) -- (353,234.67) -- (323.5,234.67) -- cycle ;
%Shape: Rectangle [id:dp48320277504224607] 
\draw   (353,204.92) -- (382.5,204.92) -- (382.5,234.67) -- (353,234.67) -- cycle ;
%Shape: Rectangle [id:dp40501271297605923] 
\draw   (382.5,204.92) -- (412,204.92) -- (412,234.67) -- (382.5,234.67) -- cycle ;
%Shape: Rectangle [id:dp1126108056319195] 
\draw   (412,204.92) -- (441.5,204.92) -- (441.5,234.67) -- (412,234.67) -- cycle ;
%Shape: Rectangle [id:dp6384978108368102] 
\draw   (441.5,204.92) -- (471,204.92) -- (471,234.67) -- (441.5,234.67) -- cycle ;
%Straight Lines [id:da7399777449734068] 
\draw    (390.5,122.53) -- (390.02,202.06) ;
\draw [shift={(390,205.06)}, rotate = 270.35] [fill={rgb, 255:red, 0; green, 0; blue, 0 }  ][line width=0.08]  [draw opacity=0] (8.93,-4.29) -- (0,0) -- (8.93,4.29) -- cycle    ;
%Shape: Rectangle [id:dp2970344146912638] 
\draw  [fill={rgb, 255:red, 254; green, 255; blue, 147 }  ,fill opacity=1 ] (360,52.03) -- (468.5,52.03) -- (468.5,121) -- (360,121) -- cycle ;
%Shape: Rectangle [id:dp14928679177976534] 
\draw  [fill={rgb, 255:red, 255; green, 255; blue, 255 }  ,fill opacity=1 ] (468.5,52.03) -- (577,52.03) -- (577,121) -- (468.5,121) -- cycle ;

% Text Node
\draw (95,27.9) node [anchor=north west][inner sep=0.75pt]    {$pts$};
% Text Node
\draw (50,55) node [anchor=north west][inner sep=0.75pt]   [align=left] {size\_ = 0};
% Text Node
\draw (50,74.5) node [anchor=north west][inner sep=0.75pt]   [align=left] {capacity\_ = 3};
% Text Node
\draw (49,95.5) node [anchor=north west][inner sep=0.75pt]   [align=left] {ptr\_ = 0x742578};
% Text Node
\draw (96.5,211.4) node [anchor=north west][inner sep=0.75pt]    {$x$};
% Text Node
\draw (47.5,237) node [anchor=north west][inner sep=0.75pt]   [align=left] {size\_ = 2};
% Text Node
\draw (47.5,256.5) node [anchor=north west][inner sep=0.75pt]   [align=left] {capacity\_ = 5};
% Text Node
\draw (48.5,276.5) node [anchor=north west][inner sep=0.75pt]   [align=left] {ptr\_ = 0x53d6d0};
% Text Node
\draw (84.5,5.9) node [anchor=north west][inner sep=0.75pt]    {$Stack$};
% Text Node
\draw (251.5,281.5) node [anchor=north west][inner sep=0.75pt]   [align=left] {1.0};
% Text Node
\draw (280.5,281.25) node [anchor=north west][inner sep=0.75pt]   [align=left] {1.0};
% Text Node
\draw (367,8.9) node [anchor=north west][inner sep=0.75pt]    {$Heap$};
% Text Node
\draw (256,58) node [anchor=north west][inner sep=0.75pt]  [font=\small] [align=left] {size\_ = 2};
% Text Node
\draw (256,77.5) node [anchor=north west][inner sep=0.75pt]  [font=\small] [align=left] {capacity\_ = 5};
% Text Node
\draw (257,97.5) node [anchor=north west][inner sep=0.75pt]  [font=\small] [align=left] {ptr\_ =0x1df1ab0};
% Text Node
\draw (209.5,162) node [anchor=north west][inner sep=0.75pt]   [align=left] {1.0};
% Text Node
\draw (238.5,161.75) node [anchor=north west][inner sep=0.75pt]   [align=left] {1.0};
% Text Node
\draw (87.83,315.4) node [anchor=north west][inner sep=0.75pt]    {$x+x$};
% Text Node
\draw (49.5,339.67) node [anchor=north west][inner sep=0.75pt]   [align=left] {size\_ = 2};
% Text Node
\draw (49.5,359.17) node [anchor=north west][inner sep=0.75pt]   [align=left] {capacity\_ = 5};
% Text Node
\draw (50.5,379.17) node [anchor=north west][inner sep=0.75pt]   [align=left] {ptr\_ = 0x53d6d0};
% Text Node
\draw (255.5,378.17) node [anchor=north west][inner sep=0.75pt]   [align=left] {2.0};
% Text Node
\draw (284.5,377.92) node [anchor=north west][inner sep=0.75pt]   [align=left] {2.0};
% Text Node
\draw (326.5,211.17) node [anchor=north west][inner sep=0.75pt]   [align=left] {2.0};
% Text Node
\draw (355.5,210.92) node [anchor=north west][inner sep=0.75pt]   [align=left] {2.0};
% Text Node
\draw (368,58) node [anchor=north west][inner sep=0.75pt]  [font=\small] [align=left] {size\_ = 2};
% Text Node
\draw (368,77.5) node [anchor=north west][inner sep=0.75pt]  [font=\small] [align=left] {capacity\_ = 5};
% Text Node
\draw (369,97.5) node [anchor=north west][inner sep=0.75pt]  [font=\small] [align=left] {ptr\_ =0x1df1ab8};
% Text Node
\draw (473,57) node [anchor=north west][inner sep=0.75pt]  [font=\small] [align=left] {size\_ = 0};
% Text Node
\draw (473,76.5) node [anchor=north west][inner sep=0.75pt]  [font=\small] [align=left] {capacity\_ = 0};
% Text Node
\draw (475,96.5) node [anchor=north west][inner sep=0.75pt]  [font=\small] [align=left] {ptr\_ =nullptr};
\end{tikzpicture}

*Step 3*. When `push_back(const T&)` returns, the temporary `x + x` will die and its destructor is called and the memory allocated on the heap is freed.

Our code is clearly not performing well: we create a copy of the temporary `x + x` and destroy the source of the copy immediately afterwards, which means we unnecessarily allocate and free memory that we could have just moved from source to the copy.

![Step #3](move_semantics_04.png)

With the next statement, again we insert `x` into `pts`:

```
pts.push_back(x)
```

Again, `pts` copies `x`.
